**Importamos liberías**

In [ ]:
import pandas as pd
from google.colab import files
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import io

**Leemos el archivo de datos**

In [ ]:
uploaded = files.upload()

**Guardamos en una variable la tabla leída**

In [ ]:
df2 = pd.read_csv(io.BytesIO(uploaded['ETSII-Data-01_organized.csv']), sep=';')
dataset = df2.copy()
dataset.tail()

**Ploteamos los datos de energía**

In [ ]:
sns.lineplot(x=dataset.index, y="Energia", data=dataset);

**Separamos el dataset de entrenamiento y de validación**

In [ ]:

train_dataset = dataset.sample(frac=0.8,random_state=0)
test_dataset = dataset.drop(train_dataset.index)

train_labels = train_dataset.pop('Energia')
test_labels = test_dataset.pop('Energia')

train_labels = train_labels.astype(dtype=float)
test_labels = test_labels.astype(dtype=float)

**Calculamos algunas estadísticas**

In [ ]:
train_labels_stats = train_labels.describe()

train_stats = train_dataset.describe()
train_stats = train_stats.transpose()

print(train_stats)

**Normalizamos tanto entradas como salidas**

In [48]:
def norm(x):
  return (x - train_stats['mean']) / train_stats['std']
normed_train_data = norm(train_dataset)
normed_test_data = norm(test_dataset)

def norm_(x):
  return (x - train_labels_stats['mean']) / train_labels_stats['std']
normed_train_data_labels = norm_(train_labels)
normed_test_data_labels = norm_(test_labels)

**Definimos el modelo de la red a entrenar**

In [76]:
def build_model():
  model = keras.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=[len(train_dataset.keys())]),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1)
  ])

  optimizer = tf.keras.optimizers.RMSprop(0.005)

  model.compile(loss='mse',
                optimizer=optimizer,
                metrics=['mae', 'mse'])
  return model

**Instanciamos y mostramos resumen**

In [ ]:
model = build_model()
model.summary()

**Entrenamos a la red**

In [ ]:
EPOCHS = 1000

history = model.fit(
  normed_train_data, normed_train_data_labels,
  epochs=EPOCHS, validation_split = 0.2)

**Guardamos la información del entrenamiento**

In [ ]:
hist = pd.DataFrame(history.history)
hist['epoch'] = history.epoch
hist.tail()

**Ploteamos la información del entrenamiento**

In [ ]:
def plot_history(history):
  hist = pd.DataFrame(history.history)
  hist['epoch'] = history.epoch

  plt.figure()
  plt.xlabel('Epoch')
  plt.ylabel('Mean Abs Error [Energia]')
  plt.plot(hist['epoch'], hist['mae'],
           label='Train Error')
  plt.plot(hist['epoch'], hist['val_mae'],
           label = 'Val Error')
  plt.ylim([0,1])
  plt.legend()

  plt.figure()
  plt.xlabel('Epoch')
  plt.ylabel('Mean Square Error [$Energia^2$]')
  plt.plot(hist['epoch'], hist['mse'],
           label='Train Error')
  plt.plot(hist['epoch'], hist['val_mse'],
           label = 'Val Error')
  plt.ylim([0,2])
  plt.legend()
  plt.show()


plot_history(history)

**Evaluamos el test set**

In [ ]:
loss, mae, mse = model.evaluate(normed_test_data, normed_test_data_labels, verbose=2)

print("Testing set Mean Abs Error: {:5.2f} Energy".format(mae))

**Ploteamos los resultados del test set**

In [ ]:
test_predictions = model.predict(normed_test_data).flatten()

plt.scatter(normed_test_data_labels, test_predictions)
plt.xlabel('True Values [Energía]')
plt.ylabel('Predictions [Energía]')
plt.axis('equal')
plt.axis('square')
plt.xlim([0,plt.xlim()[1]])
plt.ylim([0,plt.ylim()[1]])
_ = plt.plot([-150, 150], [-150, 150])

**Ploteamos distribución del error en forma de histograma**

In [ ]:
error = test_predictions - normed_test_data_labels
plt.hist(error, bins = 10)
plt.xlabel("Prediction Error [Energía]")
_ = plt.ylabel("Count")